In [1]:
# --- 1. Import Libraries ---
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from pointMAE import PointMAEModel  # Import your PointMAE model
import os
import random
import numpy as np
from plyfile import PlyData
import trimesh  # For loading .off files as point clouds
import time
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm  # tqdm for Jupyter notebooks

In [2]:
# --- 2. Define Hyperparameters ---
BATCH_SIZE = 32
EPOCHS = 300
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.05
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
CHECKPOINT_DIR = "./checkpoints_modelnet10"
PRETRAINED_CHECKPOINT = "./checkpoints_pointmae/pointmae_epoch_50.pth"
LOG_DIR = "./tensorboard_logs_modelnet10"

# Create directories if they don't exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
print("Device: ", DEVICE)

# Initialize TensorBoard writer
writer = SummaryWriter(log_dir=LOG_DIR)

Device:  cuda


In [3]:
# --- 3. Define the ModelNet10 Dataset ---
class ModelNet10Dataset(Dataset):
    def __init__(self, root_dir, split='train', random_split=False, num_points=1024, seed=42, augment=False):
        """
        ModelNet10 dataset class.

        Args:
            root_dir (str): Root directory containing ModelNet10.
            split (str): 'train' or 'test'.
            random_split (bool): If True, use random data split instead of the preset split.
            num_points (int): Number of points to sample from each point cloud.
            seed (int): Random seed for reproducibility.
            augment (bool): Apply data augmentation if True.
        """
        self.root_dir = root_dir
        self.split = split
        self.num_points = num_points
        self.augment = augment
        self.data = []
        
        # Set up random split if specified
        if random_split:
            self.random_seed_split(seed)
        else:
            self.preset_split()
    
    def preset_split(self):
        """Use preset train/test split from ModelNet10."""
        classes = sorted(os.listdir(self.root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        for cls_name in classes:
            class_path = os.path.join(self.root_dir, cls_name, self.split)
            for file_name in os.listdir(class_path):
                if file_name.endswith('.off'):
                    self.data.append((os.path.join(class_path, file_name), self.class_to_idx[cls_name]))
    
    def random_seed_split(self, seed):
        """Create a random split by shuffling files."""
        random.seed(seed)
        classes = sorted(os.listdir(self.root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        for cls_name in classes:
            class_files = os.listdir(os.path.join(self.root_dir, cls_name))
            random.shuffle(class_files)
            split_idx = int(len(class_files) * 0.8)
            if self.split == 'train':
                self.data.extend([(os.path.join(self.root_dir, cls_name, f), self.class_to_idx[cls_name]) for f in class_files[:split_idx]])
            else:
                self.data.extend([(os.path.join(self.root_dir, cls_name, f), self.class_to_idx[cls_name]) for f in class_files[split_idx:]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path, class_idx = self.data[idx]
        mesh = trimesh.load(file_path)
        points = mesh.sample(self.num_points)
        points = np.array(points, dtype=np.float32)
        
        # Apply augmentation if specified
        if self.augment and self.split == 'train':
            points = self.apply_augmentations(points)
        
        return torch.tensor(points, dtype=torch.float32), class_idx

    def apply_augmentations(self, points):
        """Random scaling and translation."""
        scale = np.random.uniform(0.8, 1.2)
        points *= scale
        points += np.random.uniform(-0.1, 0.1, size=(1, 3))
        return points


# --- 4. Initialize Dataloaders ---
root_dir = './ModelNet10'
train_dataset = ModelNet10Dataset(root_dir, split='train', random_split=False, augment=True)
test_dataset = ModelNet10Dataset(root_dir, split='test', random_split=False, augment=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# --- 5. Load Pretrained Model and Set Up Classifier ---
# Load pretrained PointMAE model
token_dim = 256
num_heads = 8
num_layers = 6
num_patches = 64
num_pts_per_patch = 32
num_channels = 3
mask_ratio = 0.65

# Initialize PointMAE Model
pointmae_model = PointMAEModel(
    input_dim=num_channels,
    token_dim=token_dim,
    num_heads=num_heads,
    num_layers=num_layers,
    mask_ratio=mask_ratio,
    num_patches=num_patches,
    num_pts_per_patch=num_pts_per_patch,
    finetune=True, # Disable masking 
).to(DEVICE)
pointmae_model.load_state_dict(torch.load(PRETRAINED_CHECKPOINT, weights_only=True))
encoder = pointmae_model.encoder

class EncoderWithClassifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(token_dim * 3, num_classes, bias=False)  # token_dim * 3 due to concatenation

    def forward(self, x):
        # Encode the input to get tokens
        encoded_tokens, *_ = self.encoder(x)  # Shape: (B, num_patches, token_dim)
        
        # Extract CLS token (first token)
        cls_token = encoded_tokens[:, 0, :]  # Shape: (B, token_dim)

        # Mean pooling across all tokens
        mean_pooled = encoded_tokens.mean(dim=1)  # Shape: (B, token_dim)

        # Max pooling across all tokens
        max_pooled, _ = encoded_tokens.max(dim=1)  # Shape: (B, token_dim)

        # Concatenate CLS token, mean-pooled, and max-pooled features
        combined_features = torch.cat([cls_token, mean_pooled, max_pooled], dim=-1)  # Shape: (B, token_dim * 3)

        # Pass through the classifier head
        logits = self.classifier(combined_features)  # Shape: (B, num_classes)
        return logits

classification_model = EncoderWithClassifier(encoder, NUM_CLASSES).to(DEVICE)

In [ ]:
# --- Calculate class distribution for cross entropy class weights
# --- MAY TAKE A WHILE
# from collections import Counter
# import torch

# # Step 1: Initialize a counter
# class_counts = Counter()

# # Step 2: Count the occurrences of each class in the training dataset
# for _, label in train_dataset:
#     class_counts[label] += 1

# # Step 3: Convert counts to a list
# num_classes = len(class_counts)
# class_count_list = [class_counts[i] for i in range(num_classes)]
class_count_list = [106, 515, 889, 200, 200, 465, 200, 680, 392, 344] # Precomputed 

# Step 4: Compute class weights (optional, for use in CrossEntropyLoss)
total_samples = sum(class_count_list)
class_weights = [total_samples / count for count in class_count_list]
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32, device=DEVICE)

print("Class Counts:", class_count_list)
print("Class Weights:", class_weights)


In [5]:
# --- 6. Define Optimizer and Scheduler ---
optimizer = optim.AdamW(classification_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=0)

In [ ]:
# --- 7. Define Training Loop with tqdm ---
def train_one_epoch(model, loader, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(loader), total=len(loader), desc=f"Epoch [{epoch+1}/{EPOCHS}]")

    # Initialize CrossEntropy loss
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    
    for batch_idx, (points, labels) in progress_bar:
        points, labels = points.to(DEVICE), labels.to(DEVICE)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update metrics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update tqdm progress bar
        progress_bar.set_postfix(Batch=f"{batch_idx+1}/{len(loader)}", Loss=f"{loss.item():.4f}")

    avg_loss = running_loss / len(loader)
    accuracy = correct / total
    return avg_loss, accuracy


def validate_one_epoch(model, loader, epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    # Initialize CrossEntropy loss
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    with torch.no_grad():
        progress_bar = tqdm(enumerate(loader), total=len(loader), desc=f"Validation Epoch [{epoch+1}/{EPOCHS}]")
        for batch_idx, (points, labels) in progress_bar:
            points, labels = points.to(DEVICE), labels.to(DEVICE)
            outputs = model(points)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Update tqdm progress bar
            progress_bar.set_postfix(Batch=f"{batch_idx+1}/{len(loader)}", Loss=f"{loss.item():.4f}")

    avg_loss = running_loss / len(loader)
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
# --- 8. Training and Validation Loop ---
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(classification_model, train_loader, optimizer, epoch)
    val_loss, val_acc = validate_one_epoch(classification_model, test_loader, epoch)
    
    # Step the scheduler
    scheduler.step()
    
    # Log metrics to TensorBoard
    writer.add_scalar("Train/Loss", train_loss, epoch)
    writer.add_scalar("Train/Accuracy", train_acc, epoch)
    writer.add_scalar("Validation/Loss", val_loss, epoch)
    writer.add_scalar("Validation/Accuracy", val_acc, epoch)
    writer.add_scalar("Learning_Rate", scheduler.get_last_lr()[0], epoch)

    # Print epoch stats
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Save checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        checkpoint_path = os.path.join(CHECKPOINT_DIR, f"classification_epoch_{epoch+1}.pth")
        torch.save(classification_model.state_dict(), checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

# Close TensorBoard writer
writer.close()
print("Training complete.")

Epoch [1/300]:   0%|          | 0/125 [00:00<?, ?it/s]

Validation Epoch [1/300]:   0%|          | 0/29 [00:00<?, ?it/s]

Epoch [1/300] - Train Loss: 2.0523, Train Acc: 0.3287, Val Loss: 4.1535, Val Acc: 0.1443


Epoch [2/300]:   0%|          | 0/125 [00:00<?, ?it/s]

Validation Epoch [2/300]:   0%|          | 0/29 [00:00<?, ?it/s]

...

Epoch [125/300] - Train Loss: 0.1440, Train Acc: 0.9466, Val Loss: 1.6155, Val Acc: 0.5551


Epoch [126/300]:   0%|          | 0/125 [00:00<?, ?it/s]

Validation Epoch [126/300]:   0%|          | 0/29 [00:00<?, ?it/s]

Epoch [126/300] - Train Loss: 0.1354, Train Acc: 0.9504, Val Loss: 1.3099, Val Acc: 0.6674


Epoch [127/300]:   0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os

# --- Load Classification Model Checkpoint ---
def load_classification_checkpoint(model, checkpoint_path):
    if os.path.isfile(checkpoint_path):
        print(f"Loading checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=DEVICE, weights_only=True)
        model.load_state_dict(checkpoint['model_state_dict'])
        print("Checkpoint loaded successfully.")
    else:
        print(f"Checkpoint not found at {checkpoint_path}")


# --- Evaluate Model on Test Set with Voting ---
def evaluate_model_with_voting(model, loader, num_votes=10, class_names=None):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for points, labels in tqdm(loader, desc="Evaluating Test Set with Voting"):
            points, labels = points.to(DEVICE), labels.to(DEVICE)
            batch_size = points.size(0)
            vote_predictions = []

            for v in range(num_votes):
                # Apply random augmentations to the points
                augmented_points = apply_test_augmentations(points.clone())

                # Forward pass
                outputs = model(augmented_points)
                _, predicted = outputs.max(1)
                vote_predictions.append(predicted.cpu().numpy())

            # Majority voting
            vote_predictions = np.array(vote_predictions)  # Shape: (num_votes, batch_size)
            final_predictions = []

            for i in range(batch_size):
                votes = vote_predictions[:, i]
                most_common = Counter(votes).most_common(1)[0][0]
                final_predictions.append(most_common)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(final_predictions)

    # Calculate accuracy and F1 score
    test_accuracy = accuracy_score(all_labels, all_predictions)
    test_f1_score = f1_score(all_labels, all_predictions, average='weighted')

    # Calculate and optionally display confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_predictions)

    if class_names is not None:
        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.show()
    else:
        print("Confusion Matrix:")
        print(conf_matrix)

    return test_accuracy, test_f1_score, conf_matrix

def apply_test_augmentations(points):
    """Apply random rotations and jittering for test-time augmentation."""
    return points

In [ ]:
# --- Main Evaluation Loop ---
checkpoint_epochs = [10, 50, 100, 150, 200, 250, 300]
results = []
checkpoint_epochs.reverse()

for epoch in checkpoint_epochs:
    checkpoint_path = f"./checkpoints_modelnet10/pointMAE/classification_epoch_{epoch}.pth"
    print(f"\nEvaluating Model at Epoch {epoch}...")
    
    try:
        load_classification_checkpoint(classification_model, checkpoint_path)
        test_accuracy_voting, test_f1_score_voting, conf_matrix = evaluate_model_with_voting(
            classification_model, test_loader, num_votes=10, class_names=class_names
        )
        print(f"Accuracy: {test_accuracy_voting}; F1 Score: {test_f1_score_voting};")
        results.append((epoch, test_accuracy_voting, test_f1_score_voting))
    except FileNotFoundError as e:
        print(e)

# --- Print Results ---
print("\nEvaluation Results:")
print("Epoch\tAccuracy\tF1 Score")
for epoch, acc, f1 in results:
    print(f"{epoch}\t{acc:.4f}\t\t{f1:.4f}")


Evaluating Model at Epoch 300...
Loading checkpoint from ./checkpoints_modelnet10/pointMAE/classification_epoch_300.pth
Checkpoint loaded successfully.


Evaluating Test Set with Voting:   0%|          | 0/29 [00:00<?, ?it/s]

Sample 0:
  Ground Truth Label: 0
  All Votes: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Final Predicted Label: 0
  Logits for each vote:
    Vote 1: [16.072721481323242, 1.318589210510254, 0.5162745714187622, -4.142401218414307, -1.9251630306243896, -2.3471555709838867, -5.9330315589904785, 7.889880180358887, -3.0701513290405273, 1.450615406036377]
    Vote 2: [12.520066261291504, 0.40597298741340637, 1.1744698286056519, -1.5440210103988647, -2.3747520446777344, -2.385354518890381, -5.520094394683838, 6.291991233825684, -2.9728991985321045, 2.176534414291382]
    Vote 3: [13.44953727722168, -0.31106075644493103, 0.06192340701818466, -2.171377420425415, -1.5399562120437622, -4.731729030609131, -6.087968349456787, 7.145591735839844, -1.4955238103866577, 2.899047613143921]
    Vote 4: [13.30393123626709, -0.39053499698638916, 0.011323891580104828, -1.9113855361938477, -0.14556366205215454, -5.590342998504639, -4.749540328979492, 6.2675347328186035, -4.117166996002197, 4.67055082321167]
    Vote 5

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4b4b59b400>>
Traceback (most recent call last):
  File "/nfs/cc-filer/home/gunnerstone/anaconda3/envs/pytorch3d/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Sample 0:
  Ground Truth Label: 1
  All Votes: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  Final Predicted Label: 1
  Logits for each vote:
    Vote 1: [-2.6291213035583496, 16.562864303588867, 2.132089138031006, -4.431171894073486, -4.668832302093506, -7.400076389312744, -3.311612844467163, 1.1647213697433472, -0.3293469548225403, -2.843830108642578]
    Vote 2: [-1.3213013410568237, 13.648956298828125, 0.09726982563734055, -3.803231954574585, -4.930740833282471, -8.563074111938477, -3.7335405349731445, 4.762251853942871, 0.22135798633098602, -2.351173162460327]
    Vote 3: [-2.873011827468872, 17.615039825439453, 2.225921630859375, -4.943271636962891, -3.5974481105804443, -6.322399616241455, -2.2186484336853027, -2.1537373065948486, -0.4421980381011963, -2.3646631240844727]
    Vote 4: [-2.893077850341797, 16.89979362487793, -1.0037014484405518, -4.130988597869873, -3.5741539001464844, -6.65397310256958, -1.8938419818878174, 2.1402060985565186, -1.2123368978500366, -3.1303656101226807]
    Vote

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4b4b59b400>>
Traceback (most recent call last):
  File "/nfs/cc-filer/home/gunnerstone/anaconda3/envs/pytorch3d/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 